In [ ]:
# 記事反映
import requests
import json
from wordpress_xmlrpc.methods import media
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import media, posts
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc import Client, WordPressPost
from PIL import Image, ImageDraw, ImageFont
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import os
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
import urllib
from bs4 import BeautifulSoup
import re
import csv
import codecs
import urllib.request as req
import glob
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import rcParams
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import timedelta
import datetime
from dotenv import load_dotenv
from selenium.webdriver.common.by import By

from sqlalchemy import (Column, String, Text, ForeignKey, \
                create_engine, MetaData, DECIMAL, DATETIME, exc, event, Index)
from sqlalchemy.schema import UniqueConstraint
from sqlalchemy.orm import (sessionmaker, relationship, scoped_session)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.mysql import INTEGER as Integer
from datetime import datetime
from sqlalchemy import Column, Integer, create_engine, MetaData, DATETIME, String,Date,Float
from sqlalchemy.orm import sessionmaker
from pathlib import Path
import datetime
from sqlalchemy import create_engine, text
from  src.utils import *
print('ライブラリの読み込み完了')

# .envファイルの内容を読み込見込む
load_dotenv()

def login_scraping_site(area_name):
    global browser
    options = Options()
    #options.add_argument('--headless')
    options.add_argument("--no-sandbox")
    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)#ChromeDriverManager().install()
    browser.implicitly_wait(10)
    url_login = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/login_form_mail"
    #admageを開く
    browser.get(url_login)
    browser.implicitly_wait(10)
    # id
    element = browser.find_element(By.NAME,"id")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_ID'))
    # pw
    element = browser.find_element(By.NAME,"pass")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_PW'))
    browser.implicitly_wait(10)
    element = browser.find_element(By.CLASS_NAME,"box_hole_view_report_input")
    element.click()
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/select_area"
    browser.get(url)
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/?area={area_name}"
    browser.get(url)
    browser.implicitly_wait(10)
    return browser

target_date:str = '2023-03-21'
prefecture_number = '23'
browser = login_scraping_site('chubu')
#エリアの都道府県の名前と対応番号を取得
prefectures_box_elements = browser.find_element(By.CLASS_NAME,"one_sp_block")
prefectures_elements:list = prefectures_box_elements.find_elements(By.CLASS_NAME,"mgn_serch_list_bottom")
prefecture_name_and_number_dict:dict[str:str] = {}

for element in prefectures_elements:
    prefecture_name:str = element.find_element(By.CLASS_NAME,"side_bar_L_O").text
    #print(prefecture_name)
    link_url:str = element.find_element(By.TAG_NAME,'a').get_attribute('href')
    prefecture_number:int = link_url.split('ken=')[-1]
    #print(prefecture_number)
    prefecture_name_and_number_dict[prefecture_name] = prefecture_number
prefecture_name_and_number_dict

In [81]:
prefecture_name_and_number_dict

{'愛知県': '23',
 '静岡県': '22',
 '山梨県': '19',
 '長野県': '20',
 '岐阜県': '21',
 '新潟県': '15',
 '富山県': '16',
 '石川県': '17',
 '福井県': '18',
 '三重県': '24'}

In [ ]:
i = 0
#取材取得日指定
target_date :datetime = datetime.date.today() + datetime.timedelta(days=1)

#合成用の空のDF
furture_syuzai_list_df = pd.DataFrame(index=[], columns=['都道府県','イベント日','店舗名','取材名','取材ランク'])


for prefecture_name,prefecture_number in prefecture_name_and_number_dict.items():
    print(prefecture_name)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/osusume_list?ken={prefecture_number}&ymd={target_date}"
    browser.get(url)
    browser.implicitly_wait(10)
    kiji_element_box = browser.find_element(By.CLASS_NAME,"osusume_list_container")
    kiji_elements_list:list = kiji_element_box.find_elements(By.CLASS_NAME,"osbox")
    for kiji_number,kiji_element in enumerate(kiji_elements_list):
        if 'プレミアム会員登録' == browser.find_element(By.CLASS_NAME,"menu_child").text:
            browser = login_scraping_site('chubu')
            url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/osusume_list?ken={prefecture_number}&ymd={target_date}"
            browser.get(url)
            browser.implicitly_wait(10)
        else:
            pass
        future_syuzai_element_box = browser.find_element(By.CLASS_NAME,"osusume_list_container")
        syuzai_record_elements = future_syuzai_element_box.find_elements(By.CLASS_NAME,"inner_osbox")
        #break
        for i,syuzai_record_element in enumerate(syuzai_record_elements):
            #print(syuzai_record_element.text)
            syuzai_record_element_list = syuzai_record_element.find_elements(By.CLASS_NAME,"list_event_name")
            print(syuzai_record_element_list)
            if len(syuzai_record_element_list) == 0:
                continue
            tenpo_name = syuzai_record_element.find_element(By.CLASS_NAME,"oslh2").text
            for syuzai_name_element in syuzai_record_element_list:
                #print(syuzai_name_element.text)
                syuzai_rank = syuzai_name_element.find_element(By.CLASS_NAME,"list_event_name_rank").text
                schedule_name = syuzai_name_element.find_element(By.CLASS_NAME,"list_event_name_li").text
                print(tenpo_name,syuzai_rank,schedule_name)
                record = pd.Series([prefecture_name,target_date, tenpo_name,schedule_name,syuzai_rank], index=furture_syuzai_list_df.columns)
                furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
    break
#browser.quit()
i = 0
furture_syuzai_list_df = pd.DataFrame(index=[], columns=['都道府県','イベント日','店舗名','取材名','取材ランク'])
for prefecture_name,prefecture_number in prefecture_name_and_number_dict.items():
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/osusume_list?ken={prefecture_number}&ymd={target_date}"
    browser.get(url)
    browser.implicitly_wait(10)
    kiji_element_box = browser.find_element(By.CLASS_NAME,"osusume_list_container")
    kiji_elements_list:list = kiji_element_box.find_elements(By.CLASS_NAME,"osbox")
    for kiji_number,kiji_element in enumerate(kiji_elements_list):
        if 'プレミアム会員登録' == browser.find_element(By.CLASS_NAME,"menu_child").text:
            browser = login_scraping_site('chubu')
            url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/osusume_list?ken={prefecture_number}&ymd={target_date}"
            browser.get(url)
            browser.implicitly_wait(10)
        else:
            pass
        future_syuzai_element_box = browser.find_element(By.CLASS_NAME,"osusume_list_container")
        syuzai_record_elements = future_syuzai_element_box.find_elements(By.CLASS_NAME,"inner_osbox")
        #break
        for i,syuzai_record_element in enumerate(syuzai_record_elements):
            #print(syuzai_record_element.text)
            syuzai_record_element_list = syuzai_record_element.find_elements(By.CLASS_NAME,"list_event_name")
            print(syuzai_record_element_list)
            if len(syuzai_record_element_list) == 0:
                continue
            tenpo_name = syuzai_record_element.find_element(By.CLASS_NAME,"oslh2").text
            for syuzai_name_element in syuzai_record_element_list:
                #print(syuzai_name_element.text)
                syuzai_rank = syuzai_name_element.find_element(By.CLASS_NAME,"list_event_name_rank").text
                schedule_name = syuzai_name_element.find_element(By.CLASS_NAME,"list_event_name_li").text
                print(tenpo_name,syuzai_rank,schedule_name)
                record = pd.Series([prefecture_name,target_date, tenpo_name,schedule_name,syuzai_rank], index=furture_syuzai_list_df.columns)
                furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
    break
#browser.quit()

In [2]:
# -*- coding: utf-8 -*-


#try:
# WORDPRESS_ID = os.environ['WORDPRESS_ID']
# WORDPRESS_PW = os.environ['WORDPRESS_PW']
# WORDPRESS_URL = os.environ['WORDPRESS_URL']
# wp = Client(WORDPRESS_URL, WORDPRESS_ID, WORDPRESS_PW)

win_path = 'image/全媒体/'
# 埼玉千葉神奈川記事反映本番用
wp_completed_footer = ''
for ken_number in ['23']:
    read_image_jpg = f"image/全媒体/tokyo_ika_board.jpg"
    font_path = "font/LightNovelPOPv2.otf"

    options = Options()
    # options.add_argument('--headless')
    # options.add_argument("--no-sandbox") 
    #options.add_argument('--user-agent=' + 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36')
    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)

    url_login = "https://hall-navi.com/login_form_mail"
    browser.implicitly_wait(10)

    browser.get(url_login)
    browser.implicitly_wait(10)
    # id
    element = browser.find_element(By.NAME, "id")
    element.click()
    browser.implicitly_wait(10)
    element.send_keys('tsc953u@gmail.com')
    # pw
    element = browser.find_element(By.NAME, "pass")
    element.click()
    browser.implicitly_wait(10)
    element.send_keys('jdhibh')
    browser.implicitly_wait(10)
    element = browser.find_element(By.CLASS_NAME,"box_hole_view_report_input")
    element.click()
    time.sleep(1)

    browser.refresh()
    todouhuken_number = 1

    today = datetime.date.today()
    tomorrow = datetime.date.today() + datetime.timedelta(days=1)

    # for baitai in ['スロパチ','天草ヤスヲ','ホールサーチマン','Gooパチ','ホール攻略']:
    #     tokyou_syuzai_df_a = tokyou_syuzai_df[tokyou_syuzai_df['baitai'] == baitai]
    #     previous_syuzai = ''p
    #     print('\n'  ,baitai)
    #     for syuzai in tokyou_syuzai_df_a.itertuples():
    #             #syuzai_list.append(syuzai)
    #         if syuzai[1] != previous_syuzai:
    #             print('\n\n■',syuzai[1],'\n   ★',syuzai[2])

    month_tomorrow_str = tomorrow.strftime("%m月").lstrip('0')
    day_tomorrow_str = tomorrow.strftime("%d日").lstrip('0')
    tomorrow_str = month_tomorrow_str + day_tomorrow_str
    tomorrow_url = tomorrow.strftime("%Y-%m-%d")
    output_path = f"eva_board_{tomorrow_url}.jpg"

    url = f"https://hall-navi.com/?area=chubu"
    browser.get(url)
    url = f"https://hall-navi.com/osusume_list?ken={ken_number}&ymd={tomorrow_url}"

    browser.get(url)
    # ログイン後にhtmlを取得する
    html = browser.page_source.encode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    kiji_list = soup.find_all(class_='osbox_out view_osbox')
    time.sleep(1)
    #browser.quit()
    elems = soup.select('.sche_list_contents.osle')
    todouhuken_str = soup.find('h1').text.replace('｜おすすめ総合ランキング', '').replace('｜', '  ')
    h1_text = soup.find('h1').text.replace('｜おすすめ総合ランキング', '').replace('｜', '  ')
    todoufuken_kanji = h1_text.split('  ')[0]

    parlar_name_list:list[str] = []
    syuzai_name:list[str] = []
    tenpo_only_text = []
    chousa_syuzai = []
    context = ''
    ichitenpo_syuzai = []
    #syuzai_df()
    for kiji in kiji_list:
        syuzai_list = kiji.find_all(class_='list_event_name_li')
        if len(syuzai_list) == 0:
            continue
        tenpo_name = kiji.find(class_='oslh2').text
        tenpo_name = tenpo_name.replace('・', '').replace('\n', '').replace(' ', '').replace('　', '')
        print(tenpo_name)
        parlar_name_list.append(tenpo_name)

# except Exception as e:
#     print(e)
#browser.quit()

読み込み完了
ライブラリの読み込み完了


C:\Users\tsc95\AppData\Local\Temp\ipykernel_31280\3786244716.py:78: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)


キクヤ春日井店
MEGAコンコルド豊川インター店
メガコンコルド1111BLAZE店
メガコンコルド800春日井店+
ミカド一宮インター店
メガコンコルド1177大口41号通り店
MEGAコンコルド1280稲沢店
リブレ尾張瀬戸店
M&K岡崎店
bingo21
ZENT木曽川店
ZENT豊田本店
ZENT稲沢店
キャッスル大金店
タイキ稲沢駅前店
メガコンコルド1220みなと23号通り店
ラッキー一番江南店
コンコルド800一宮尾西インター店
ミカド稲沢店
ラッキー1BAN日進竹の山店
マルハン中川店
がちゃぽん蟹江店
ZENT下市場店
タイキ藤沢店
オーギヤ西尾店
ラッキー一番上飯田店
yout小牧店
プレイランド名宝
キング観光サウザンド今池2号店
ラッキープラザ名古屋西インター七宝店
メガスロットコンコルド345吉浜店
KYORAKU西店
ZENT扶桑店
大一岩塚店
京楽会館豊明店
マリオン長良橋西店
プレイランド第一平和


In [3]:
parlar_name_prefecture_num_dict :dict[str:str]= {'愛知県':'23','静岡県':'22','山梨県':\
    '19','長野県':'20','岐阜県':'21','新潟県':'15','富山県':'16','石川県':'17','福井県':'18','三重県':'24'}

In [4]:
parlar_name_list

['キクヤ春日井店',
 'MEGAコンコルド豊川インター店',
 'メガコンコルド1111BLAZE店',
 'メガコンコルド800春日井店+',
 'ミカド一宮インター店',
 'メガコンコルド1177大口41号通り店',
 'MEGAコンコルド1280稲沢店',
 'リブレ尾張瀬戸店',
 'M&K岡崎店',
 'bingo21',
 'ZENT木曽川店',
 'ZENT豊田本店',
 'ZENT稲沢店',
 'キャッスル大金店',
 'タイキ稲沢駅前店',
 'メガコンコルド1220みなと23号通り店',
 'ラッキー一番江南店',
 'コンコルド800一宮尾西インター店',
 'ミカド稲沢店',
 'ラッキー1BAN日進竹の山店',
 'マルハン中川店',
 'がちゃぽん蟹江店',
 'ZENT下市場店',
 'タイキ藤沢店',
 'オーギヤ西尾店',
 'ラッキー一番上飯田店',
 'yout小牧店',
 'プレイランド名宝',
 'キング観光サウザンド今池2号店',
 'ラッキープラザ名古屋西インター七宝店',
 'メガスロットコンコルド345吉浜店',
 'KYORAKU西店',
 'ZENT扶桑店',
 '大一岩塚店',
 '京楽会館豊明店',
 'マリオン長良橋西店',
 'プレイランド第一平和']

In [5]:
username = os.getenv('DB_USER_NAME')
password=os.getenv('DB_PASSWORD')
host=os.getenv('DB_HOST')
port='3306'
database=os.getenv('DB_NAME')
db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}"

# DB接続するためのEngineインスタンス
engine = create_engine(db_url, echo=False)

# Sessionの作成
session = scoped_session(sessionmaker(autocommit = False,autoflush = False,bind = engine))
# modelで使用する
Base = declarative_base()
Base.query = session.query_property()

twitter = UtilsTwitterClass()
print(vars(twitter))

query = f"select * from schedule where 店舗名 = 'キクヤ春日井店' AND イベント日 = '{twitter.tomorrow_date_string_sql}'"
df = pd.read_sql_query(sql=text(query), con=engine.connect())
df.sort_values('イベント日')

C:\Users\tsc95\AppData\Local\Temp\ipykernel_31280\3057314543.py:14: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


{'tomorrow_8numbers_string': '20230321', 'tomorrow_date_stinrg_jp': '3/21(火)', 'tomorrow_date_string_sql': '2023-03-21', 'yesterday': datetime.date(2023, 3, 19), 'tomorrow': datetime.date(2023, 3, 21), 'yesterday_date_string': '3/19(日)', 'id': '', 'pw': '', 'tweet_text': '', 'image_path_list': [], 'main_tweet_text': '', 'project_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site', 'image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image', 'tweet_footer_text': '\n\n#XXXX 20230319\n\n', 'created_image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image\\image', 'servise_account_json_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\config\\twitteranalytics-jsonsercretkey.json'}


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,682,愛知県,2023-03-21,火,キクヤ春日井店,シーサ。オフミー取材,オフミー,A,2023-03-15 11:05:52
1,2793,愛知県,2023-03-21,火,キクヤ春日井店,シーサ。オフミー取材,オフミー,A,2023-03-16 11:05:27
2,4984,愛知県,2023-03-21,火,キクヤ春日井店,ナビ子AI予想【差枚プラス】★1 (毎週 火曜日),ホールナビ,A,2023-03-18 11:06:06


In [6]:
pledge_df = pd.read_csv(r'csv\取材公約用マスター.csv')
pledge_df
convert_parlar_name_df = pd.read_csv(r'csv\店舗名変換テーブル.csv')
convert_parlar_name_df = convert_parlar_name_df.rename(columns={'ホールナビ店舗名':'店舗名'})
convert_parlar_name_df

,都道府県,店舗kintone,shop_id,店舗名,DMM店舗名,アナスロ店舗名,緯度,経度,住所
0,北海道,P01378,1378,アミューズメントパーラー吉兆,アミューズメントパーラー吉兆,アミューズメントパーラー吉兆,43.085828,141.514724,北海道江別市大麻泉町47-37
1,北海道,P01582,1582,ひまわり音更店,音更ひまわり,ひまわり音更店,42.961369,143.207331,北海道河東郡音更町木野大通東13丁目1番21
2,北海道,P01351,13160,EBH,EBH,EBH,43.106287,141.292820,北海道札幌市西区発寒15条4-13-1
3,茨城県,P02562,2562,キコーナ水戸駅南店,キコーナ水戸駅南店,キコーナ水戸駅南店,36.365522,140.477010,茨城県水戸市城南1-4-1
4,神奈川県,P04933,4933,ピーズ・タウン桜ヶ丘,P’Stown桜ヶ丘,ピーズ・タウン桜ヶ丘,35.451103,139.465162,神奈川県大和市福田1-6-10
...,...,...,...,...,...,...,...,...,...
2352,東京都,P00363,363,楽園渋谷駅前店,楽園渋谷駅前店,楽園渋谷駅前店,35.658285,139.699800,東京都渋谷区道玄坂1-3-1
2353,山梨県,P12714,13138,マルハン甲府店,マルハン甲府店,マルハン甲府店,35.631724,138.553588,山梨県甲府市国母8-20-10
2354,東京都,P12738,13170,メッセ西荻窪店,メッセ西荻窪店,メッセ西荻窪店,35.703498,139.600920,東京都杉並区西荻南3-14-7
2355,埼玉県,P12792,13227,マルキ戸田店,マルキ戸田店,マルキ戸田店,NaN,NaN,埼玉県戸田市美女木東1-3-2


In [15]:
import folium
import branca
# 地図生成
folium_map = folium.Map(location=[35.1814, 136.9063], zoom_start=15)

# マーカープロット（ポップアップ設定，色変更，アイコン変更）
text = ''
tomorrow_schedule_df = pd.DataFrame(index=[], columns=[])
for parlar_name in parlar_name_list[:10]:
    query = f"select * from schedule WHERE 店舗名 = '{parlar_name}' AND イベント日 = '{twitter.tomorrow_date_string_sql}'"
    print(query)
    extract_parlar_name_df = pd.read_sql_query(sql=query, con=engine.connect())
    extract_parlar_name_df =  extract_parlar_name_df[~extract_parlar_name_df['取材名'].str.contains('ナビ子')]
    extract_parlar_name_df = extract_parlar_name_df[~extract_parlar_name_df.duplicated(subset=['イベント日','曜日','店舗名','取材名','媒体名'])]
    display(extract_parlar_name_df)
    tomorrow_schedule_df = pd.concat([tomorrow_schedule_df, extract_parlar_name_df])
    #display(convert_parlar_name_df)
    #tenpo_syuzai_merge_df = pd.merge(extract_parlar_name_df, convert_parlar_name_df, on='店舗名')
    # merge_df = pd.merge(tenpo_syuzai_merge_df, pledge_df, on='取材名')
    # merged_df =merge_df[merge_df['都道府県_x'] == '愛知県'].sort_values('取材ランク')
tomorrow_schedule_df
tenpo_syuzai_merge_df = pd.merge(tomorrow_schedule_df, convert_parlar_name_df, on='店舗名')
#tenpo_syuzai_merge_df
merge_df = pd.merge(tomorrow_schedule_df,pledge_df, on='取材名')
merge_df

select * from schedule WHERE 店舗名 = 'キクヤ春日井店' AND イベント日 = '2023-03-21'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,682,愛知県,2023-03-21,火,キクヤ春日井店,シーサ。オフミー取材,オフミー,A,2023-03-15 11:05:52


select * from schedule WHERE 店舗名 = 'MEGAコンコルド豊川インター店' AND イベント日 = '2023-03-21'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
1,3826,愛知県,2023-03-21,火,MEGAコンコルド豊川インター店,ガチアンケート金色,ガチアン,A,2023-03-17 11:05:46


select * from schedule WHERE 店舗名 = 'メガコンコルド1111BLAZE店' AND イベント日 = '2023-03-21'


...

select * from schedule WHERE 店舗名 = 'M&K岡崎店' AND イベント日 = '2023-03-21'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,292,愛知県,2023-03-21,火,M&K岡崎店,赤羽六華来店,その他取材・企画,・,2023-03-15 11:05:52


select * from schedule WHERE 店舗名 = 'bingo21' AND イベント日 = '2023-03-21'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,272,愛知県,2023-03-21,火,bingo21,ズバスロ,あかだ主任のガチンコ…,・,2023-03-15 11:05:52
1,293,愛知県,2023-03-21,火,bingo21,ちりめんじゃこ太さん来店,その他取材・企画,B,2023-03-15 11:05:52
2,1184,愛知県,2023-03-21,火,bingo21,スロッター×スロッター,パチンコ店長のホール攻略,・,2023-03-15 11:05:52
3,2523,愛知県,2023-03-21,火,bingo21,旧イベ(1のつく日),旧イベ,・,2023-03-15 11:05:52


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名_x,取材ランク,取得時間,媒体名_y,公約内容
0,682,愛知県,2023-03-21,火,キクヤ春日井店,シーサ。オフミー取材,オフミー,A,2023-03-15 11:05:52,オフミー,NaN
1,1181,愛知県,2023-03-21,火,メガコンコルド1111BLAZE店,サプライズスロッター,パチンコ店長のホール攻略,A,2023-03-15 11:05:52,パチンコ店長のホール攻略,NaN
2,2519,愛知県,2023-03-21,火,メガコンコルド1111BLAZE店,旧イベ(1のつく日),旧イベ,・,2023-03-15 11:05:52,旧イベ,NaN
3,2523,愛知県,2023-03-21,火,bingo21,旧イベ(1のつく日),旧イベ,・,2023-03-15 11:05:52,旧イベ,NaN
4,925,愛知県,2023-03-21,火,メガコンコルド800春日井店+,潜入取材ギガテン,スロパチステーション,S,2023-03-15 11:05:52,スロパチステーション,未調査
5,1196,愛知県,2023-03-21,火,ミカド一宮インター店,マキシマムスロッター,パチンコ店長のホール攻略,B,2023-03-15 11:05:52,パチンコ店長のホール攻略,NaN
6,361,愛知県,2023-03-21,火,メガコンコルド1177大口41号通り店,打ちたい台が固まっているか聞かせてくれ！プレミアム,でちゃう！,・,2023-03-15 11:05:52,でちゃう！,NaN
7,882,愛知県,2023-03-21,火,メガコンコルド1177大口41号通り店,番組収録(ジャンバリ),ジャンバリ.TV,B,2023-03-15 11:05:52,ジャンバリ.TV,NaN
8,997,愛知県,2023-03-21,火,MEGAコンコルド1280稲沢店,その日、最も盛り上がった光景を取材する,パチンコウォーカー,・,2023-03-15 11:05:52,パチンコウォーカー,NaN
9,292,愛知県,2023-03-21,火,M&K岡崎店,赤羽六華来店,その他取材・企画,・,2023-03-15 11:05:52,その他取材・企画,NaN


In [77]:
html_table_df = merge_df[['イベント日','曜日','店舗名','取材名','媒体名_x','公約内容']]
html_table_df['イベント日'] = twitter.tomorrow_date_stinrg_jp
html_table_df = html_table_df[['イベント日','店舗名','取材名','媒体名_x','公約内容']]
print(html_table_df.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>イベント日</th>
      <th>店舗名</th>
      <th>取材名</th>
      <th>媒体名_x</th>
      <th>公約内容</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>3/21(火)</td>
      <td>キクヤ春日井店</td>
      <td>シーサ。オフミー取材</td>
      <td>オフミー</td>
      <td>未調査</td>
    </tr>
    <tr>
      <td>3/21(火)</td>
      <td>メガコンコルド1111BLAZE店</td>
      <td>サプライズスロッター</td>
      <td>パチンコ店長のホール攻略</td>
      <td>未調査</td>
    </tr>
    <tr>
      <td>3/21(火)</td>
      <td>メガコンコルド1111BLAZE店</td>
      <td>旧イベ(1のつく日)</td>
      <td>旧イベ</td>
      <td>未調査</td>
    </tr>
    <tr>
      <td>3/21(火)</td>
      <td>bingo21</td>
      <td>旧イベ(1のつく日)</td>
      <td>旧イベ</td>
      <td>未調査</td>
    </tr>
    <tr>
      <td>3/21(火)</td>
      <td>メガコンコルド800春日井店+</td>
      <td>潜入取材ギガテン</td>
      <td>スロパチステーション</td>
      <td>未調査</td>
    </tr>
    <tr>
      <td>3/21(火)</td>
      <td>ミカド一宮インター店</td>
      <td>マキシマムスロッター</td>
      <td>パチンコ

C:\Users\tsc95\AppData\Local\Temp\ipykernel_31280\3171230412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  html_table_df['イベント日'] = twitter.tomorrow_date_stinrg_jp


In [82]:
def recommend_image(write_image_context, length):
    global context_1, save_image_jpg,output_path

    # 元画像を読み込んでくる
    save_image_jpg = fr"image\temp_image\eva_board_{prefecture_name}_{tomorrow_url}.jpg"
    read_image_path = r"image\image_source\eva_board.jpg"  #

    image = Image.open(read_image_path)
    draw = ImageDraw.Draw(image)

    # フォントを指定
    font_path = r"font\LightNovelPOPv2.otf"
    font = ImageFont.truetype(font_path, size=40)

    # 文字を描く
    # 最初の(0,0)は文字の描画を開f始する座標位置です　もちろん、(10,10)などでもOK
    # fillはRGBで文字の色を決めています
    draw.multiline_text((55, 40), write_image_context, fill=(255, 255, 255), font=font, spacing=10, stroke_width=5, stroke_fill=(55, 55, 55))

    # 画像を保存する
    image.save(save_image_jpg)
    print('length', length)
    length_croped = 110 + length * 66  # 500
    im = Image.open(save_image_jpg)
    im_crop = im.crop((0, 0, 1000, length_croped))
    im_crop.save(save_image_jpg, quality=100)

In [83]:
prefecture_name = '愛知県'
title = f'{prefecture_name} {twitter.tomorrow_date_stinrg_jp} 明日のスロットイベントまとめ\n\n'
schedule_text = ''
last_time_parlar_name = ''
for i,row in merge_df.iterrows():
    #print(row)
    if last_time_parlar_name != row['店舗名']:
        schedule_text += row['店舗名'] + '\n'
    else:
        pass
    schedule_text += '　☆' + row['取材名'] + '\n'

    if row['取材名'].startswith(tuple(['旧イベ'])):
        continue

    if str(row['公約内容']) == 'nan':
        schedule_text += '       ┗' + '未調査' + '\n'
    else:
        schedule_text += '       ┗' + row['公約内容'] + '\n'


    last_time_parlar_name = row['店舗名']

input_image_text : str = title + schedule_text
text_length = input_image_text.count('\n')
print('length', text_length)
print(input_image_text)
recommend_image(input_image_text, text_length)

length 35
愛知県 3/21(火) 明日のスロットイベントまとめ

キクヤ春日井店
　☆シーサ。オフミー取材
       ┗未調査
メガコンコルド1111BLAZE店
　☆サプライズスロッター
       ┗未調査
　☆旧イベ(1のつく日)
bingo21
　☆旧イベ(1のつく日)
メガコンコルド800春日井店+
　☆潜入取材ギガテン
       ┗未調査
ミカド一宮インター店
　☆マキシマムスロッター
       ┗未調査
メガコンコルド1177大口41号通り店
　☆打ちたい台が固まっているか聞かせてくれ！プレミアム
       ┗未調査
　☆番組収録(ジャンバリ)
       ┗未調査
MEGAコンコルド1280稲沢店
　☆その日、最も盛り上がった光景を取材する
       ┗未調査
M&K岡崎店
　☆赤羽六華来店
       ┗未調査
bingo21
　☆ズバスロ
       ┗未調査
　☆ちりめんじゃこ太さん来店
       ┗未調査
　☆スロッター×スロッター
       ┗4つの公約が存在。要HPチェック

length 35
